# Week 3 Assignment
## Segmenting and Clustering Neighborhoods in Toronto

#### Exploring and visualising neighbourhoods

In [1]:
# install required packages
import pandas as pd
import numpy as np

#conda install -c conda-forge geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#conda install -c conda-forge folium
import folium

import requests # library to handle requests

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

##### Load data

In [2]:
# load stored dataframe with post codes/boroughs/neighbourhoods from wikipedia scraping
%store -r df_2

print(df_2.head())

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_2['Borough'].unique()),
        df_2.shape[0]
    )
)

  PostalCode           Borough                                Neighbourhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  
The dataframe has 10 boroughs and 103 neighborhoods.


##### Build map of toronto

In [3]:
# get location of map centre
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [4]:
# create map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_2['Latitude'], df_2['Longitude'], df_2['Borough'], df_2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

##### Use foursquare to get venues for each neighbourhood

In [5]:
# foursquare credentials
CLIENT_ID = 'JJRBC2IVQIWJSBV2QZIRRIP0E4I3RPAUR2BPQTGSB3RLYNQ1' # your Foursquare ID
CLIENT_SECRET = 'BYH2BP1SKA4QQMQ24J0NTZBMW0FF5HIN33C4KHHKL5EV3N2J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JJRBC2IVQIWJSBV2QZIRRIP0E4I3RPAUR2BPQTGSB3RLYNQ1
CLIENT_SECRET:BYH2BP1SKA4QQMQ24J0NTZBMW0FF5HIN33C4KHHKL5EV3N2J


In [6]:
# function to collect venues for all neighbourhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


## test function
#getNearbyVenues(["Parkwoods", "Victoria Village"], [43.753259, 43.725882], [-79.329656,-79.315572])

In [7]:
# build dataframe of venues in all neighbourhoods

Toronto_venues = getNearbyVenues(names=df_2['Neighbourhood'],
                                   latitudes=df_2['Latitude'],
                                   longitudes=df_2['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

##### Explore venue dataset

In [8]:
print(Toronto_venues.shape)
print(Toronto_venues.head())
Toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories of venues.'.format(len(Toronto_venues['Venue Category'].unique())))

(2122, 7)
       Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2         Parkwoods              43.753259              -79.329656   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1         Brookbanks Pool       43.751389       -79.332184   
2           Variety Store       43.751974       -79.333114   
3  Victoria Village Arena       43.723481       -79.315635   
4               Portugril       43.725819       -79.312785   

          Venue Category  
0                   Park  
1                   Pool  
2      Food & Drink Shop  
3           Hockey Arena  
4  Portuguese Restaurant  
There are 267 uniques categories o

##### Analyse each neighbourhood to find most frequent venue types

In [9]:
# one hot encoding by venue types
print("List of venues: ", Toronto_venues.shape)
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print("One hot encoded venues: ", Toronto_onehot.shape)

# note: it appears "Neighborhood" is a category of venue
print(any(Toronto_onehot.columns == 'Neighborhood'))
print(Toronto_onehot['Neighborhood'].unique())

# add neighbourhood column back to dataframe using UK spelling
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighborhood']

# move neighborhood column to the first column
Toronto_onehot = Toronto_onehot[ ['Neighbourhood'] + [ col for col in Toronto_onehot.columns if col != 'Neighbourhood' ] ]

print(Toronto_onehot.head())
print(Toronto_onehot.shape)

List of venues:  (2122, 7)
One hot encoded venues:  (2122, 267)
True
[0 1]
      Neighbourhood  Accessories Store  Adult Boutique  Afghan Restaurant  \
0         Parkwoods                  0               0                  0   
1         Parkwoods                  0               0                  0   
2         Parkwoods                  0               0                  0   
3  Victoria Village                  0               0                  0   
4  Victoria Village                  0               0                  0   

   Airport  Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0        0                   0             0               0                0   
1        0                   0             0               0                0   
2        0                   0             0               0                0   
3        0                   0             0               0                0   
4        0                   0             0             

In [10]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
hoods = Toronto_venues['Neighborhood']
print(Toronto_grouped.shape)
Toronto_grouped.head()

(96, 268)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# find most common types of venue in each neighbourhood

# function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# set how many types of venues to consider
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Pub,Diner,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Supermarket,Mobile Phone Shop,Sushi Restaurant,Ice Cream Shop,Shopping Mall,Restaurant,Deli / Bodega
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Restaurant,Café,Sushi Restaurant,Pub,Liquor Store,Thai Restaurant


##### K-means clustering of neighbourhoods

In [16]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)
#print(Toronto_grouped_clustering.describe())
#print(Toronto_grouped_clustering.values)

# standardised the means of the dummy variables, but no material impact on clusters
sc_X = StandardScaler()
data_clust = sc_X.fit_transform(Toronto_grouped_clustering)
#print(data_clust.describe())

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(data_clust)

# check cluster labels generated for each row in the dataframe
print("Cluster labels:", kmeans.labels_)

Cluster labels: [4 4 4 4 4 0 4 4 4 3 4 4 4 0 4 2 4 4 0 4 4 4 4 1 4 4 4 4 0 0 4 0 4 4 4 0 4
 4 4 4 4 4 4 0 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 0 4 4 4 4 4 4 4 0 4
 4 0 4 4 4 4 4 4 4 0 0 4 4 4 4 4 4 4 4 4 4 4]


In [17]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

Toronto_merged = neighborhoods_venues_sorted

# merge Toronto_grouped with Toronto main data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(df_2.set_index('Neighbourhood'), on='Neighbourhood')
Toronto_merged.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,4,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,M1S,Scarborough,43.794200,-79.262029
1,4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Pub,Diner,Department Store,Dessert Shop,M8W,Etobicoke,43.602414,-79.543484
2,4,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Supermarket,Mobile Phone Shop,Sushi Restaurant,Ice Cream Shop,Shopping Mall,Restaurant,Deli / Bodega,M3H,North York,43.754328,-79.442259
3,4,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M2K,North York,43.786947,-79.385975
4,4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Restaurant,Café,Sushi Restaurant,Pub,Liquor Store,Thai Restaurant,M5M,North York,43.733283,-79.419750


In [18]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], 
                                  Toronto_merged['Longitude'], 
                                  Toronto_merged['Neighbourhood'], 
                                  Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining the clusters that were generated

In [19]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
5,0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Beer Bar,Pharmacy,Seafood Restaurant,Bakery,Restaurant,Jazz Club,M5E,Downtown Toronto,43.644771,-79.373306
13,0,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant,Juice Bar,M5G,Downtown Toronto,43.657952,-79.387383
18,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant,M5L,Downtown Toronto,43.648198,-79.379817
28,0,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Japanese Restaurant,Electronics Store,Chinese Restaurant,Food Court,Shoe Store,Toy / Game Store,M2J,North York,43.778517,-79.346556
29,0,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Asian Restaurant,M5X,Downtown Toronto,43.648429,-79.382280
31,0,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Diner,Theater,Bookstore,M5B,Downtown Toronto,43.657162,-79.378937
35,0,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Restaurant,Brewery,Fried Chicken Joint,Italian Restaurant,Baseball Stadium,M5J,Downtown Toronto,43.640816,-79.381752
43,0,"Kensington Market, Chinatown, Grange Park",Café,Vegetarian / Vegan Restaurant,Mexican Restaurant,Coffee Shop,Vietnamese Restaurant,Dessert Shop,Comfort Food Restaurant,Grocery Store,Bar,Bakery,M5T,Downtown Toronto,43.653206,-79.400049
48,0,"Little Portugal, Trinity",Bar,Coffee Shop,Restaurant,Vietnamese Restaurant,Café,Vegetarian / Vegan Restaurant,Asian Restaurant,Men's Store,Cupcake Shop,Brewery,M6J,West Toronto,43.647927,-79.419750
62,0,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Park,M7A,Downtown Toronto,43.662301,-79.389494


##### Cluster 0: downtown neighbourhoods, with a focus on coffee shops, bars, and restaurants

In [20]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
23,1,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Light Rail Station,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Deli / Bodega,M1P,Scarborough,43.75741,-79.273304


##### Cluster 1: a specific surburban neighbourhood characterised by asian reastauraunts

In [21]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
15,2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Hotel,Café,Mediterranean Restaurant,Men's Store,M4Y,Downtown Toronto,43.66586,-79.38316


##### Cluster 2: a specific downtown neighbourhood similar to others in Cluster 0 except for the comparatively high prevelance of gay bars

In [22]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
9,3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Rental Car Location,Plane,Sculpture Garden,Harbor / Marina,Bar,Airport Gate,M5V,Downtown Toronto,43.628947,-79.39442


##### Cluster 3: The airport neighbourhood, containing airport specific venues

In [23]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,4,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,M1S,Scarborough,43.794200,-79.262029
1,4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Pub,Diner,Department Store,Dessert Shop,M8W,Etobicoke,43.602414,-79.543484
2,4,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Supermarket,Mobile Phone Shop,Sushi Restaurant,Ice Cream Shop,Shopping Mall,Restaurant,Deli / Bodega,M3H,North York,43.754328,-79.442259
3,4,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M2K,North York,43.786947,-79.385975
4,4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Restaurant,Café,Sushi Restaurant,Pub,Liquor Store,Thai Restaurant,M5M,North York,43.733283,-79.419750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,4,"Willowdale, Willowdale West",Pharmacy,Discount Store,Pizza Place,Coffee Shop,Grocery Store,Gift Shop,Cupcake Shop,Drugstore,Donut Shop,Doner Restaurant,M2R,North York,43.782736,-79.442259
92,4,Woburn,Coffee Shop,Pharmacy,Korean BBQ Restaurant,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Department Store,Donut Shop,M1G,Scarborough,43.770992,-79.216917
93,4,Woodbine Heights,Park,Beer Store,Dance Studio,Curling Ice,Skating Rink,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,M4C,East York,43.695344,-79.318389
94,4,York Mills West,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,M2P,North York,43.752758,-79.400049


##### Cluster 4: most suburban areas, containing a mix of restaurants, stores, and leisure activities